# Tensorflow Transformer (Base) Implementation 0x80
----

Well, I tried to understand the transformers in full detail but without "using" them in practice, but in theory. In this notebook I will simply reimplement/recreate the Tensorflow example transformer (https://www.tensorflow.org/tutorials/text/transformer) and later try to train it on my data, rather than to understand it first and then implement the transformer by myself from scratch (There is no value in doing so and it makes things even harder). 

I was way too much into the idea, to not use other peoples code and develop it on my own, but what I forgot is, that this ting is already invented and was implemented multiple times. When I started my project there weren't that many implementation out there, which were easy enough to understand, they were rather optimized for speed and did lots of tricks to achieve that. That made understanding the code so much harder, that I sticked to the idea to have to implement a transformer as simple as possible. But now thankfully such simple transformers exists (even as a tutorial). So there is no need to stick to this idea any further.

I think it should be clear, that it is better to start with something that works (a fully implemented transformer) and then to experiment with it and then improve and modify it. As already stated, other transformer implementations were somehow too complicated and specialized, rather than simple. The Tensorflow transformer is simple enough to reimplement and to extend for own purposes.

So the original code is licensed under Apache 2.0 License.

`
Except as otherwise noted, the content of this page is licensed under the Creative Commons Attribution 4.0 License, and code samples are licensed under the Apache 2.0 License. For details, see the Google Developers Site Policies. Java is a registered trademark of Oracle and/or its affiliates.
`

This code may still be rewritten and/or refactored later if it works. This python notebook should be seen as my way to start a transformer for my future experiments.

Main of the code is not mine, but i made some modifications to it. For the original code please refer to the transformer tutorial website of tensorflow mentioned above.

So here we go...

In [ ]:
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


## Preparing the input pipeline
----
We follow this example for a protugese to english translation. So we prepare a well known/prepared dataset.

In [ ]:
import os
import json


In [ ]:
# Length of a "sentence"
MAX_LENGTH = 64

# ??
BUFFER_SIZE = 50000

# Number of sentences processed in one batch
BATCH_SIZE = 256

In [ ]:

strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    
    # load the context dataset
    context_dataset = tf.data.TextLineDataset( 
        os.path.join('D:\\Downloads\\Big-Code-excerpt','NextLineTranslationDataset.jsonl.from'))

    # load the prediction dataset
    nextline_dataset = tf.data.TextLineDataset(
        os.path.join('D:\\Downloads\\Big-Code-excerpt','NextLineTranslationDataset.jsonl.to'))

    
    # [16273] is the first element outside of the vocabulary, ans serves as a start element.
    # [0] serves as a padding element
    def my_json_decode(source, target):
        source_decoded = [16273] + json.loads(source.numpy()) + [0] + [0]
        target_decoded = [16273] + json.loads(target.numpy()) + [0] + [0]
        return source_decoded, target_decoded

    def tf_context_nextline_json_decode(context, nextline):
        result_context, result_nextline = tf.py_function(my_json_decode, [context, nextline], [tf.int64, tf.int64])
        result_context.set_shape([None])
        result_nextline.set_shape([None])

        return result_context, result_nextline

    def filter_max_length(x, y, max_length=MAX_LENGTH):
        return tf.logical_and(tf.size(x) <= max_length,
                            tf.size(y) <= max_length)

    # combine both datasets in a parallel corpus
    train_dataset_ = tf.data.Dataset.zip((context_dataset, nextline_dataset))
    # transform from string to bpe encoded message
    train_dataset__ = train_dataset_.map(tf_context_nextline_json_decode)

    # filter dataset entries exceeding the capacity
    train_dataset = train_dataset__.filter(filter_max_length)

    # now do preprocessing and shuffle data around
    train_dataset = train_dataset.cache()
    train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    

    def get_angles(pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2*(i//2)) / np.float32(d_model))
        return pos * angle_rates

    def positional_encoding(position, d_model):
        angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                              np.arange(d_model)[np.newaxis, :],
                              d_model)

        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]

        return tf.cast(pos_encoding, dtype=tf.float32)


    pos_encoding = positional_encoding(50,512)
    print(pos_encoding.shape)


    def create_padding_mask(seq):
        # this will create a mask from the input, whereever the input is Zero, it is treated as a padding.
        # and a one is written to the result, otherwise a Zero is written to the array (where true -> '1.0': else '0.0')
        seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

        # Mask has dimensions (batchsize, 1,1, seq_len)
        return seq[:, tf.newaxis, tf.newaxis, :]

    def create_look_ahead_mask(size):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask  # (seq_len, seq_len)


    def scaled_dot_product_attention(q, k, v, mask):
        matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

        # scale matmul_qk
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        # add the mask to the scaled tensor.
        if mask is not None:
            scaled_attention_logits += (mask * -1e9)  

        # softmax is normalized on the last axis (seq_len_k) so that the scores
        # add up to 1.
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

        output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

        return output, attention_weights

    class MultiHeadAttention(tf.keras.layers.Layer):
        def __init__(self, d_model, num_heads):
            super(MultiHeadAttention, self).__init__()
            self.num_heads = num_heads
            self.d_model = d_model

            assert d_model % self.num_heads == 0

            self.depth = d_model // self.num_heads

            self.wq = tf.keras.layers.Dense(d_model)
            self.wk = tf.keras.layers.Dense(d_model)
            self.wv = tf.keras.layers.Dense(d_model)

            self.dense = tf.keras.layers.Dense(d_model)

        def split_heads(self, x, batch_size):
            """Split the last dimension into (num_heads, depth).
            Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
            """
            x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
            return tf.transpose(x, perm=[0, 2, 1, 3])

        def call(self, v, k, q, mask):
            batch_size = tf.shape(q)[0]

            q = self.wq(q)  # (batch_size, seq_len, d_model)
            k = self.wk(k)  # (batch_size, seq_len, d_model)
            v = self.wv(v)  # (batch_size, seq_len, d_model)

            q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
            k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
            v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

            # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
            # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
            scaled_attention, attention_weights = scaled_dot_product_attention(
                q, k, v, mask)

            scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

            concat_attention = tf.reshape(scaled_attention, 
                                          (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

            output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

            return output, attention_weights

    def point_wise_feed_forward_network(d_model, dff):
        return tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])

    class EncoderLayer(tf.keras.layers.Layer):
        def __init__(self, d_model, num_heads, dff, rate=0.1):
            super(EncoderLayer, self).__init__()

            self.mha = MultiHeadAttention(d_model, num_heads)
            self.ffn = point_wise_feed_forward_network(d_model, dff)

            self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

            self.dropout1 = tf.keras.layers.Dropout(rate)
            self.dropout2 = tf.keras.layers.Dropout(rate)

        def call(self,x,training,mask):
            # first sublayer
            attn_output,_ = self.mha(x,x,x,mask)
            attn_output = self.dropout1(attn_output, training=training)
            out1 = self.layernorm1(x+attn_output)

            # second sublayer
            ffn_output = self.ffn(out1)
            ffn_output = self.dropout2(ffn_output, training=training)
            out2 = self.layernorm2(out1 + ffn_output)

            return out2


    class DecoderLayer( tf.keras.layers.Layer ):
        def __init__(self, d_model, num_heads, dff, rate = 0.1):
            super(DecoderLayer, self).__init__()

            self.mha1 = MultiHeadAttention(d_model, num_heads)
            self.mha2 = MultiHeadAttention(d_model, num_heads)

            self.ffn = point_wise_feed_forward_network(d_model, dff)

            self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
            self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

            self.dropout1 = tf.keras.layers.Dropout(rate)
            self.dropout2 = tf.keras.layers.Dropout(rate)
            self.dropout3 = tf.keras.layers.Dropout(rate)

        def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
            # first sublayer
            attn1, attn_weights_block1 = self.mha1(x,x,x,look_ahead_mask)
            attn1 = self.dropout1(attn1, training=training)
            out1 = self.layernorm1(attn1 + x)

            # second sublayer
            attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
            attn2 = self.dropout2(attn2, training=training)
            out2 = self.layernorm2(attn2 + out1)

            # third sublayer
            ffn_output = self.ffn(out2)
            ffn_output = self.dropout3(ffn_output, training=training)
            out3 = self.layernorm3(ffn_output + out2)

            return out3, attn_weights_block1, attn_weights_block2

    class Encoder(tf.keras.layers.Layer):
        def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
            super(Encoder, self).__init__()

            self.d_model = d_model
            self.num_layers = num_layers

            self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
            self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

            self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
            self.dropout = tf.keras.layers.Dropout(rate)

        def call(self, x, training, mask):
            seq_len = tf.shape(x)[1]

            x = self.embedding(x)
            x*= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
            x+= self.pos_encoding[:, :seq_len, :]

            x= self.dropout(x, training = training)

            for i in range(self.num_layers):
                x = self.enc_layers[i](x, training, mask)

            return x

    class Decoder(tf.keras.layers.Layer):
        def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
            super(Decoder, self).__init__()

            self.d_model = d_model
            self.num_layers = num_layers

            self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
            self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

            self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
            self.dropout = tf.keras.layers.Dropout(rate)

        def call(self,x,enc_output, training, look_ahead_mask, padding_mask):
            seq_length= tf.shape(x)[1]
            attention_weights = {}

            # batchsize, sequence_length, d_model
            x  =self.embedding(x)
            x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
            x +=self.pos_encoding[:, :seq_length, :]

            x= self.dropout(x, training=training)

            for i in range(self.num_layers):
                x,  block1, block2 = self.dec_layers[i](x,enc_output, training, look_ahead_mask, padding_mask)
                attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
                attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

            return x, attention_weights


    class Transformer(tf.keras.Model):
        def __init__(self, num_layers, d_model, num_heads, dff, 
                     input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
            super(Transformer, self).__init__()

            self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)
            self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

            self.final_layer = tf.keras.layers.Dense(target_vocab_size)

        def call(self, inp, tar, training, 
                 enc_padding_mask, look_ahead_mask, dec_padding_mask):
            # batch_size, inp_seq_length, d_model
            enc_output = self.encoder(inp, training, enc_padding_mask)
            dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

            final_output = self.final_layer(dec_output)

            return final_output, attention_weights


    num_layers = 3
    d_model = 512
    dff=1536
    num_heads = 8

    bpe_encoder_vocab_size = 16272
    ## +2 because of padding and masking
    input_vocab_size = bpe_encoder_vocab_size + 2
    target_vocab_size = bpe_encoder_vocab_size + 2


    #input_vocab_size = tokenizer_pt.vocab_size + 2
    #target_vocab_size = tokenizer_en.vocab_size + 2


    dropout_rate = 0.09

    class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, d_model, warmup_steps=4000):
            super(CustomSchedule, self).__init__()

            self.d_model=d_model
            self.d_model = tf.cast(self.d_model, tf.float32)

            self.warmup_steps = warmup_steps

        def __call__(self, step):
            arg1 = tf.math.rsqrt(step)
            arg2 = step * (self.warmup_steps ** -1.5 )

            return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    learning_rate = CustomSchedule(d_model)
    optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)


    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

    def loss_function(real, pred):
        mask=tf.math.logical_not(tf.math.equal(real,0))

        loss_ = loss_object(real, pred)
        mask = tf.cast(mask, dtype=loss_.dtype)

        loss_ *= mask
        return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

    transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size,
                         pe_input=input_vocab_size, pe_target=target_vocab_size,
                         rate=dropout_rate)


    def create_masks(inp, tar):
        # encoder padding mask
        enc_padding_mask = create_padding_mask(inp)

        # wird im second attentionblock im decoder benutzt, um den input zu maskieren
        dec_padding_mask = create_padding_mask(inp)

        look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
        dec_target_padding_mask = create_padding_mask(tar)
        combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

        return enc_padding_mask, combined_mask, dec_padding_mask

    checkpoint_path = "./checkpoints/train"
    ckpt = tf.train.Checkpoint( transformer = transformer, optimizer = optimizer)
    ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

    if ckpt_manager.latest_checkpoint:
        ckpt.restore(ckpt_manager.latest_checkpoint)
        print('Latest checkpoint restored!')


    EPOCHS = 10

    train_step_signature = [
        tf.TensorSpec(shape=(None,None), dtype=tf.int64),
        tf.TensorSpec(shape=(None,None), dtype=tf.int64)
    ]

    @tf.function(input_signature = train_step_signature)
    def train_step(inp, tar):
        tar_inp = tar[:, :-1]
        tar_real = tar[:, 1:]

        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

        with tf.GradientTape() as tape:
            predictions, _ = transformer(inp, tar_inp, True, enc_padding_mask, combined_mask, dec_padding_mask)
            loss = loss_function(tar_real, predictions)

            gradients = tape.gradient(loss, transformer.trainable_variables)
            optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

        train_loss(loss)
        train_accuracy(tar_real, predictions)




    for epoch in range(EPOCHS):
        start = time.time()

        train_loss.reset_states()
        train_accuracy.reset_states()

        for (batch, (inp, tar)) in enumerate(train_dataset):
            train_step(inp,tar)

            if batch%100 ==0:
                print('Epoch{} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
                    epoch+1, batch, train_loss.result(), train_accuracy.result()
                ))

        if (epoch+1)%2 == 0:
            ckpt_save_path = ckpt_manager.save()
            print('saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))

        print('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch+1, train_loss.result(), train_accuracy.result()))

        print('Time taken for one epoch: {} secs\n'.format(time.time()-start))

## Evaluation of the training using a validation dataset
----


In [ ]:
def evaluate(inp_sentence):
    start_token = [tokenizer_pt.vocab_size]
    end_token = [tokenizer_pt.vocab_size + 1]
  
    # inp sentence is portuguese, hence adding the start and end token
    inp_sentence = start_token + tokenizer_pt.encode(inp_sentence) + end_token
    encoder_input = tf.expand_dims(inp_sentence, 0)
  
    # as the target is english, the first word to the transformer should be the
    # english start token.
    decoder_input = [tokenizer_en.vocab_size]
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(MAX_LENGTH):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
            encoder_input, output)
  
        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer(encoder_input, 
                                                     output,
                                                     False,
                                                     enc_padding_mask,
                                                     combined_mask,
                                                     dec_padding_mask)
    
        # select the last word from the seq_len dimension
        predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
        # return the result if the predicted_id is equal to the end token
        if predicted_id == tokenizer_en.vocab_size+1:
            return tf.squeeze(output, axis=0), attention_weights
    
        # concatentate the predicted_id to the output which is given to the decoder
        # as its input.
        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights

In [ ]:
def plot_attention_weights(attention, sentence, result, layer):
    fig = plt.figure(figsize=(16, 8))

    sentence = tokenizer_pt.encode(sentence)

    attention = tf.squeeze(attention[layer], axis=0)

    for head in range(attention.shape[0]):
        ax = fig.add_subplot(2, 4, head+1)

        # plot the attention weights
        ax.matshow(attention[head][:-1, :], cmap='viridis')

        fontdict = {'fontsize': 10}

        ax.set_xticks(range(len(sentence)+2))
        ax.set_yticks(range(len(result)))

        ax.set_ylim(len(result)-1.5, -0.5)

        ax.set_xticklabels(
            ['<start>']+[tokenizer_pt.decode([i]) for i in sentence]+['<end>'], 
            fontdict=fontdict, rotation=90)

        ax.set_yticklabels([tokenizer_en.decode([i]) for i in result 
                            if i < tokenizer_en.vocab_size], 
                           fontdict=fontdict)

        ax.set_xlabel('Head {}'.format(head+1))

    plt.tight_layout()
    plt.show()


def translate(sentence, plot=''):
    result, attention_weights = evaluate(sentence)
    predicted_sentence = tokenizer_en.decode([i for i in result 
                                              if i<tokenizer_en.vocab_size])
    
    print('Input: {}'.format(sentence))
    print('Predicted Translation: {}'.format(predicted_sentence))
    
    if plot:
        plot_attention_weights(attention_weights, sentence, result, plot)
        
#translate("este é um problema que temos que resolver.")
#print ("Real translation: this is a problem we have to solve .")


In [ ]:
#translate("este é o primeiro livro que eu fiz.", plot='decoder_layer6_block2')
#print ("Real translation: this is the first book i've ever done.")
